# 问题1：写出get_response()


In [115]:
# 判断pat是否以？开头，并且pat中所有的都是字母
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

In [116]:
# 寻找pattern中的？P 并返回它本身saying中与之对应的部分组成的元祖，这个？P可能有多个，返回他们组成的列表
def pat_match(pattern, saying):
    if not pattern or not saying: return []
    
    if is_variable(pattern[0]):
        return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])
    else:
        if pattern[0] != saying[0]: return []
        else:
            return pat_match(pattern[1:], saying[1:])

In [117]:
is_variable('?hhh')

True

In [118]:
is_variable('hhhh')

False

In [119]:
is_variable('?hh123')

False

In [120]:
pat_match('greater than ?Y'.split(), 'hhhhhh'.split())

[]

In [121]:
pat_match('?X greater than ?Y'.split(), '3 greater than 2'.split())

[('?X', '3'), ('?Y', '2')]

In [122]:
# 将得到的list结果转化成字典
def pat_to_dic(patterns):
    return {k: v for k, v in patterns}

In [123]:
# 返回rule这一句话，并把其中的？P以及？X等等转化成我们定义的规则下相应的单词
def subsitite(rule, parsed_rules):
    if not rule: return []
    
    return [parsed_rules.get(rule[0], rule[0])] + subsitite(rule[1:], parsed_rules)

In [124]:
got_patterns = pat_match('I want ?X'.split(), 'I want iPhone'.split())

In [125]:
got_patterns

[('?X', 'iPhone')]

In [126]:
subsitite('What if you mean you if got a ?X'.split(), pat_to_dic(got_patterns))

['What', 'if', 'you', 'mean', 'you', 'if', 'got', 'a', 'iPhone']

In [127]:
' '.join(subsitite('What if you mean if you got a ?X'.split(), pat_to_dic(got_patterns)))

'What if you mean if you got a iPhone'

In [128]:
join_pat = pat_match('?P needs ?X'.split(), 'John needs vacation'.split())

In [129]:
join_pat

[('?P', 'John'), ('?X', 'vacation')]

In [183]:
pat_to_dic(join_pat)

{'?P': 'John', '?X': 'vacation'}

In [184]:
pat_to_dic([('?P', '超级'), ('?X', '你')])

{'?P': '超级', '?X': '你'}

In [130]:
' '.join(subsitite('Why does ?P need ?X ?'.split(), pat_to_dic(join_pat)))

'Why does John need vacation ?'

In [131]:
defined_patterns = {
    'I need ?X': ['Image you will get ?X soon', 'Why do you need ?X ?'],
    'My ?X told me something': ['Talk about more about your ?X', 'How do you think about your ?X ?']
}

In [132]:
import random

In [133]:
# 首先应该匹配saying对应于字典中哪一个键，
# 然后通过pat_match()获取？P和其对应的词，
# 获取之后将其对应的词替换到rules中对应的value中，
# 然后随机返回其中一句，这样就形成了对话

def get_response(saying, rules):
    
    for key_pattern in rules:
        pattern = pat_match(key_pattern.split(), saying.split())
        senten = ' '.join(subsitite(key_pattern.split(), pat_to_dic(pattern)))
        
        if senten == saying:
            respon_pattern = random.choice(rules[key_pattern])
            return ' '.join(subsitite(respon_pattern.split(), pat_to_dic(pattern)))
    pass

In [134]:
get_response('I need iPhone', defined_patterns)

'Image you will get iPhone soon'

In [135]:
get_response('My mother told me something', defined_patterns)

'Talk about more about your mother'

# Segment Match


In [136]:
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

In [137]:
is_pattern_segment('?*P')

True

In [138]:
from collections import defaultdict

In [139]:
# 获取多个匹配的字符串类似上面的pat_match

fail = [True, None]

def pat_match_with_seg(pattern, saying):
    if not pattern or not saying: return []
    
    pat = pattern[0]
    
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail

In [140]:
# 获取一个匹配的最大长度的所有字符串
def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')
    
    if not rest: return (seg_pat, saying), len(saying)
    
    for i, token in enumerate(saying):
        if rest[0] == token:
            return (seg_pat, saying[:i]), i
        
    return (seg_pat, saying), len(saying)

In [141]:
segment_match('?*P is very good'.split(), 'My dog and my cat is very good'.split())

(('?P', ['My', 'dog', 'and', 'my', 'cat']), 5)

In [142]:
pat_match_with_seg('?*P is very good and ?*X'.split(), 'My dog is very good and my cat is very cute'.split())

[('?P', ['My', 'dog']), ('?X', ['my', 'cat', 'is', 'very', 'cute'])]

In [143]:
response_pair = {
    'I need ?X': ['Why do you neeed ?X'],
    'I dont like my ?X': ['What bad things did ?X do for you?']
}

In [144]:
pat_match_with_seg('I need ?*X'.split(), 'I need an iPhone'.split())

[('?X', ['an', 'iPhone'])]

In [145]:
subsitite('Why do you neeed ?X'.split(), pat_to_dic(pat_match_with_seg('I need ?*X'.split(), 'I need an iPhone'.split())))

['Why', 'do', 'you', 'neeed', ['an', 'iPhone']]

In [146]:
def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k,v in patterns}

In [147]:
subsitite('Why do you neeed ?X'.split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(), 'I need an iPhone'.split())))

['Why', 'do', 'you', 'neeed', 'an iPhone']

In [148]:
("?*X hello ?*Y", "Hi, how do you do")

('?*X hello ?*Y', 'Hi, how do you do')

In [149]:
subsitite("Hi, how do you do?".split(), pat_to_dict(pat_match_with_seg('?*X hello ?*Y'.split(), 
                  "I am mike, hello ".split())))

['Hi,', 'how', 'do', 'you', 'do?']

In [150]:
rules = {
    "?*X hello ?*Y": ["Hi, how do you do?"],
    "I was ?*X": ["Were you really ?X ?", "I already knew you were ?X ."]
}

# 问题2：改写程序以支持中文输入

In [151]:
import jieba

In [152]:
chinese_patt = '/'.join(jieba.cut('我?X你'))

In [153]:
chinese_patt

'我/?/X/你'

In [154]:
chinese_say = '/'.join(jieba.cut('我喜欢你'))

In [155]:
# 将‘?’和‘X..P’分在一起
def str_fenci(st_fenci):
    index = st_fenci.find('?')
    former_fenci = st_fenci[:index+1]
    while index != -1:
        last_fenci = ''.join(st_fenci[index + 2:])
        index = st_fenci.find('?', index+1)
        mi_fenci = st_fenci[len(former_fenci)+1:index+1]
        former_fenci += mi_fenci
    return former_fenci+last_fenci


In [156]:
pat_match_with_seg(str_fenci(chinese_patt).split('/'),chinese_say.split('/'))

[('?X', '喜欢')]

In [157]:
chin_pat_three = '/'.join(jieba.cut('我?X喜欢?P'))

In [158]:
str_fenci(chin_pat_three)

'我/?X/喜欢/?P'

In [212]:
str_fenci('/'.join(jieba.cut('妈妈?X喜欢?P')))

'妈妈/?X/喜欢/?P'

In [159]:
chin_say_two = '/'.join(jieba.cut('我超级喜欢你'))

In [160]:
chin_say_two

'我/超级/喜欢/你'

In [161]:
pat_match_with_seg(str_fenci(chin_pat_three).split('/'),chin_say_two.split('/'))

[('?X', '超级'), ('?P', '你')]

In [162]:
chin_pat_four = '/'.join(jieba.cut('韩梅梅?X喜欢?P'))

In [163]:
chin_pat_four

'韩梅梅/?/X/喜欢/?/P'

In [164]:
subsitite(str_fenci(chin_pat_four).split('/'), pat_to_dict(pat_match_with_seg(str_fenci(chin_pat_three).split('/'),chin_say_two.split('/'))))

['韩梅梅', '超级', '喜欢', '你']

In [165]:
''.join(subsitite(str_fenci(chin_pat_four).split('/'), pat_to_dict(pat_match_with_seg(str_fenci(chin_pat_three).split('/'),chin_say_two.split('/')))))

'韩梅梅超级喜欢你'

In [229]:
defined_chin_patt = {
    '我想?y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '妈妈?x喜欢?y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y']
}

In [235]:
# 中文回答
def get_chin_response(say, rules):
    
    for key_patt in rules:
        key_pattern = '/'.join(jieba.cut(key_patt))
        saying = '/'.join(jieba.cut(say))
        pattern = pat_match_with_seg(str_fenci(key_pattern).split('/'),saying.split('/'))
        senten = ''.join(subsitite(str_fenci(key_pattern).split('/'), pat_to_dic(pattern)))
        
        if senten == say:
            respon_pattern ='/'.join(jieba.cut(random.choice(rules[key_patt]))) 
            return ''.join(subsitite(str_fenci(respon_pattern).split('/'), pat_to_dic(pattern)))
    pass

In [236]:
get_chin_response('我想妈妈',defined_chin_patt)

'你可以想想你很快就可以妈妈了'

In [237]:
get_chin_response('我想妈妈',defined_chin_patt)

'你觉得妈妈有什么意义呢？'

In [238]:
get_chin_response('我想妈妈',defined_chin_patt)

'为什么你想妈妈'

In [239]:
pat_match_with_seg(str_fenci('/'.join(jieba.cut('妈妈?X喜欢?P'))).split('/'),'/'.join(jieba.cut('妈妈我喜欢水果')).split('/'))

[('?X', '我'), ('?P', '水果')]

# 问题3：


# 问题4：
